In [33]:
with open("lolbas.txt",'r', encoding='utf-8') as f:
    text = f.read()
    
    chars = sorted(set(text))
    print(chars)
    
    vocab_Size = len(chars)
    print(vocab_Size)
    
    
    ## building a map 
    
    stoi = {ch:i for i,ch in enumerate(chars)}
    Itos = {i:ch for i,ch in enumerate(chars)}
    
    encode = lambda x: [stoi[c] for c in x]
    decode = lambda x: "".join([Itos[c] for c in x])
    

    
    

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
96


In [34]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

n = int(0.9*len(data))

train_data = data[:n] ## first 90% of the data set is up to train, rest val
val_data = data[n:]

block_size  = 8 
batch_size = 4

torch.manual_seed(1337)

def get_batch(split):
    # generate a small number of data inputs x and targets Y
    
    data = train_data if split == 'train' else val_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    
    return x, y 

tensor([ 0, 47, 66, 78, 70,  1, 27,  1, 34, 69, 69, 74, 79, 54, 85, 74, 77, 15,
        70, 89, 70,  0, 36, 80, 78, 78, 66, 79, 69,  1, 27,  1, 36, 27, 61, 56,
        74, 79, 69, 80, 88, 84, 61, 46, 74, 68, 83, 80, 84, 80, 71, 85, 15, 47,
        38, 53, 61, 39, 83, 66, 78, 70, 88, 80, 83, 76, 61, 87, 21, 15, 17, 15,
        20, 17, 20, 18, 26, 61, 34, 69, 69, 74, 79, 54, 85, 74, 77, 15, 70, 89,
        70,  1, 14, 34, 69, 69, 74, 79, 51, 80])


In [35]:
xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[78, 78, 66, 79, 69,  1, 27,  1],
        [80, 71,  1, 85, 73, 70,  1, 71],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [74, 80, 79,  1, 85, 80, 80, 77]])
tensor([[78, 66, 79, 69,  1, 27,  1, 83],
        [71,  1, 85, 73, 70,  1, 71, 80],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [80, 79,  1, 85, 80, 80, 77, 15]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

# b = batch_size
# t = block_size
# c = vocab_Size

class BiogramLanguageModel(nn.Module):
    def __init__(self, vocab_Size):
        super().__init__()
        
        ## each token directly read off the logits for the next token rom a lookup table
        
        self.token_embedding_table = nn.Embedding(vocab_Size, vocab_Size)
        
    def forward(self, idx, targets=None):
        
        #idx and targets are both (b,t ) tensor of intergers 
        
        logits = self.token_embedding_table(idx) ## (b ,t, c)
        
        # pegamos a linha correspondendo na tabela do embedding e organizamos em um tensor de 3 dimensoes, btc 
        
        ## basicamente o scores
        ## oque acontece é que estamos prevendo o proximo caracter na  sequencia, com base apenas em um unico token 
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            
            logits = logits.view(B*T,C) # batch + block, vocalsize twodimensiona tensor
            targets = targets.view(B*T)
            
            
            loss = F.cross_entropy(logits,targets)
        
        return logits, loss
    
    def generate(self, idx, number_of_maxtokens):
        # idx is (B,T) array of indices in the current position 
        
        for _ in range(number_of_maxtokens):
            
            #get the predictions 
            logits, loss = self(idx)
            
            #focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            
            # apply softmax to get probabilities
            
            probs = F.softmax(logits, dim=1) # B, C
            
            # sample from the distribuition
            idx_next = torch.multinomial(probs, num_samples=1) # b,1
            
            idx = torch.cat((idx, idx_next), dim=1) # (B, T +1)
            
        return idx
    
    
m = BiogramLanguageModel(vocab_Size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), number_of_maxtokens=100)[0].tolist()))
## zero é o elemento que representa um caractere de linha nova, então é meio que razoável inserir como o primeiro caractere de uma sequência para ser a nova linha

torch.Size([32, 96])
tensor(5.4609, grad_fn=<NllLossBackward0>)

jfwi
U"CkybBdysm*f83Sda'l4{vhI;;#K}HW"?4Nn|l:sN1l5&GY*C|m#Guk?mJ0I]y;{c 7~1p/Su)RtX$SYV,G7cb5oQ%"keO


In [ ]:
### crete pytorch optimizer 

optimizer = torch.optim.AdamW(m.parameters(), lr='1e-3')

## take the gradients and update the paramets using the gradiantes 

batch_size = 32

for steps in range(100):
    ## sample batch size 
    xb, yb = get_batch('train')
